# Segmenting and clustering neightborhoods in Toronto

## Question 1

In [0]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [0]:
URL_LINK = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_page = requests.get(URL_LINK)
# html_page.content

We will use BeautifulSoup to manipulate DOM Elements of the HTML Page

In [0]:
soup = BeautifulSoup(html_page.content) #, 'html5lib')
# soup

In [0]:
html_table = soup.find('table', attrs={'class' : 'wikitable'})
# html_table

In [5]:
columns = ['Postcode', 'Borough', 'Neighborhood']
df = pd.DataFrame(columns = columns)
df

,Postcode,Borough,Neighborhood


### Populating data into the dataframe from the html table

In [6]:
df = pd.read_html(str(html_table))
print(type(df))
print(type(df[0]))
df = df[0] # for extracting the dataframe out of the list
df.columns = columns
df.head()

<class 'list'>
<class 'pandas.core.frame.DataFrame'>


,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Removing the Not assigned values of the Borough and Neighbourhood column

In [7]:
df = df[df['Borough'] != 'Not assigned']
df = df[df['Neighborhood'] != 'Not assigned']
df.head()

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [8]:
df.head(12)

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern
14,M3B,North York,Don Mills North


In [9]:
df=df.groupby("Postcode").agg(lambda x:','.join(set(x)))
df.reset_index(inplace = True)
df.head(12)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Port Union,Highland Creek,Rouge Hill"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Oakridge,Clairlea,Golden Mile"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [10]:
df.shape

(102, 3)

## Question 2

In [0]:
import requests

In [0]:
URL_GEO='https://cocl.us/Geospatial_data'

In [13]:
df_geo = pd.read_csv(URL_GEO)
df_geo.columns = ['Postcode', 'Latitude', 'Longitude']
df_geo.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [0]:
neighborhood_df = pd.merge(df, df_geo, on = 'Postcode')

In [15]:
print("Shape of merged dataframe is {}".format(neighborhood_df.shape))
neighborhood_df.head()

Shape of merged dataframe is (102, 5)


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Port Union,Highland Creek,Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Question 3

### Installing and importing necessary libraries

In [16]:
# !conda install -c conda-forge folium=0.5.0 --yes
!pip install folium

In [0]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium

### Finding co-ordinates of Torronto

In [18]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [19]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhood_df['Latitude'], neighborhood_df['Longitude'], neighborhood_df['Borough'], neighborhood_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [20]:
neighborhood_df.groupby("Borough").count()

,Postcode,Neighborhood,Latitude,Longitude
Borough,,,,
Central Toronto,9,9,9,9
Downtown Toronto,18,18,18,18
East Toronto,5,5,5,5
East York,5,5,5,5
Etobicoke,12,12,12,12
Mississauga,1,1,1,1
North York,24,24,24,24
Scarborough,17,17,17,17
West Toronto,6,6,6,6


We will first extract the dataframe that contains the name **Toronto**

In [21]:
df_toronto = neighborhood_df[neighborhood_df['Borough'].str.contains('Toronto')]
df_toronto = df_toronto.reset_index(drop = True)
df_toronto.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


### Foursquare credentials

In [22]:
CLIENT_ID = 'E2OA4IZJV5B0ZZGNP41KEP3HUU0QRZ0HSX1JDFIFCRIFW4PP' #'your-client-ID' # your Foursquare ID
CLIENT_SECRET = 'UIEK051KJZYK3LQ4VATYQYYHYEB1EIL5ITNPR5LWZLXY51MA' #'your-client-secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: E2OA4IZJV5B0ZZGNP41KEP3HUU0QRZ0HSX1JDFIFCRIFW4PP
CLIENT_SECRET:UIEK051KJZYK3LQ4VATYQYYHYEB1EIL5ITNPR5LWZLXY51MA


In [0]:
LIMIT = 30

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
toronto_venues = getNearbyVenues(names=df_toronto['Borough'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )
toronto_venues.head()

East Toronto
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
West Toronto
West Toronto
West Toronto
West Toronto
West Toronto
West Toronto
East Toronto


,Borough,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,East Toronto,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,East Toronto,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,East Toronto,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,East Toronto,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,East Toronto,43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [25]:
toronto_venues.groupby('Borough').count()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Central Toronto,107,107,107,107,107,107
Downtown Toronto,487,487,487,487,487,487
East Toronto,100,100,100,100,100,100
West Toronto,137,137,137,137,137,137


#### One hot encoding

In [26]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add borough column back to dataframe
toronto_onehot['Borough'] = toronto_venues['Borough'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Borough,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,...,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,East Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,East Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,East Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,East Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,East Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
toronto_onehot.shape

(831, 191)

In [28]:
toronto_grouped = toronto_onehot.groupby('Borough').mean().reset_index()
toronto_grouped.head()

,Borough,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,...,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.018692,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.009346,0.000000,0.009346,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.009346,0.009346,0.000000,0.000000,0.018692,0.000000,0.009346,0.000000,0.046729,0.000000,0.000000,0.000000,...,0.037383,0.000000,0.000000,0.009346,0.065421,0.000000,0.009346,0.009346,0.000000,0.00,0.000000,0.000000,0.000000,0.009346,0.000000,0.018692,0.009346,0.000000,0.00,0.00000,0.009346,0.037383,0.009346,0.000000,0.000000,0.000000,0.000000,0.009346,0.000000,0.000000,0.000000,0.009346,0.009346,0.000000,0.009346,0.000000,0.009346,0.000000,0.000000,0.009346
1,Downtown Toronto,0.002053,0.002053,0.002053,0.004107,0.00616,0.004107,0.012320,0.000000,0.002053,0.012320,0.002053,0.004107,0.002053,0.00,0.002053,0.002053,0.000000,0.028747,0.002053,0.014374,0.004107,0.016427,0.002053,0.004107,0.002053,0.006160,0.002053,0.008214,0.000000,0.008214,0.002053,0.004107,0.002053,0.000000,0.002053,0.084189,0.000000,0.006160,0.002053,...,0.034908,0.002053,0.004107,0.002053,0.010267,0.002053,0.016427,0.002053,0.002053,0.00,0.002053,0.002053,0.000000,0.006160,0.006160,0.002053,0.002053,0.000000,0.00,0.01848,0.002053,0.006160,0.000000,0.002053,0.002053,0.002053,0.012320,0.012320,0.006160,0.002053,0.002053,0.000000,0.002053,0.002053,0.016427,0.002053,0.006160,0.002053,0.000000,0.000000
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.020000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.030000,0.010000,0.000000,0.010000,0.020000,0.000000,0.000000,0.040000,0.000000,0.000000,0.010000,...,0.020000,0.000000,0.000000,0.000000,0.020000,0.000000,0.010000,0.000000,0.000000,0.01,0.000000,0.010000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.01,0.01000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000
3,West Toronto,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.007299,0.000000,0.007299,0.007299,0.021898,0.000000,0.00,0.000000,0.000000,0.000000,0.029197,0.014599,0.072993,0.000000,0.000000,0.000000,0.000000,0.000000,0.021898,0.000000,0.029197,0.014599,0.000000,0.000000,0.000000,0.014599,0.000000,0.000000,0.051095,0.007299,0.007299,0.000000,...,0.014599,0.000000,0.000000,0.007299,0.007299,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.007299,0.000000,0.007299,0.000000,0.000000,0.007299,0.00,0.00000,0.014599,0.014599,0.000000,0.000000,0.000000,0.000000,0.007299,0.007299,0.007299,0.000000,0.000000,0.000000,0.000000,0.000000,0.007299,0.000000,0.014599,0.007299,0.007299,0.007299


In [29]:
toronto_grouped.shape

(4, 191)

### Let's print each Borough along with the top 5 most common venues

In [30]:
num_top_venues = 5

for borough in toronto_grouped['Borough']:
    print("----"+borough+"----")
    temp = toronto_grouped[toronto_grouped['Borough'] == borough].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.08
1            Park  0.07
2  Sandwich Place  0.07
3            Café  0.05
4     Pizza Place  0.05


----Downtown Toronto----
         venue  freq
0         Café  0.08
1  Coffee Shop  0.08
2   Restaurant  0.03
3       Bakery  0.03
4         Park  0.03


----East Toronto----
                venue  freq
0    Greek Restaurant  0.08
1         Coffee Shop  0.05
2  Italian Restaurant  0.05
3                Café  0.04
4      Ice Cream Shop  0.04


----West Toronto----
                venue  freq
0                 Bar  0.07
1         Coffee Shop  0.05
2                Café  0.05
3         Pizza Place  0.04
4  Italian Restaurant  0.04




### Let's do clusering on the data

In [0]:
from sklearn.cluster import KMeans

In [32]:
k = 4

toronto_grouped_clustering = toronto_grouped.drop('Borough', 1)

kmeans = KMeans(n_clusters = k, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10] 

array([0, 3, 2, 1], dtype=int32)